In [1]:
import pandas as pd

# Load the dataset
df_sampled = pd.read_csv("C:/Users/Guest01/Documents/Manpreet_thesis/Datasets/Glassdoor_job_reviews2/all_reviews.csv")

# Downcast numerical columns to save memory
def downcast(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype == "int64"]
    df[float_cols] = df[float_cols].astype("float32")
    df[int_cols] = df[int_cols].astype("int32")
    return df

df_sampled = downcast(df_sampled)
df_sampled.head()


C:\Users\Guest01\AppData\Local\Temp\ipykernel_17372\4126841228.py:4: DtypeWarning: Columns (5,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sampled = pd.read_csv("C:/Users/Guest01/Documents/Manpreet_thesis/Datasets/Glassdoor_job_reviews2/all_reviews.csv")


,rating,title,status,pros,cons,advice,Recommend,CEO Approval,Business Outlook,Career Opportunities,Compensation and Benefits,Senior Management,Work/Life Balance,Culture & Values,Diversity & Inclusion,firm_link,date,job,index
0,5.0,Good,"Current Employee, more than 10 years",Knowledge gain of complete project,Financial growth and personal growth,NaN,v,o,v,3,3,3,3,3.0,3.0,Reviews/Baja-Steel-and-Fence-Reviews-E5462645.htm,"Nov 19, 2022",Manager Design,NaN
1,4.0,Good,"Former Employee, less than 1 year","Good work,good work , flexible, support","Good,work, flexible,good support, good team work",NaN,v,o,o,4,4,4,4,4.0,4.0,Reviews/Baja-Steel-and-Fence-Reviews-E5462645.htm,"Jan 29, 2022",Anonymous Employee,NaN
2,4.0,"Supervising the manufacturing the processes, e...","Current Employee, more than 1 year",This company is a best opportunity for me to l...,"Monthly Target work,Maintain production schedu...",NaN,v,o,v,2,3,2,2,2.0,2.0,Reviews/Baja-Steel-and-Fence-Reviews-E5462645.htm,"Aug 12, 2021",Production Engineer,NaN
3,1.0,terrible,"Current Employee, more than 1 year",I wish there were some to list,too many to list here,NaN,x,x,x,1.0,3.0,1.0,3.0,1.0,NaN,https://www.glassdoor.com/Reviews/Calgary-Flam...,"Sep 24, 2020",Senior Account Executive,NaN
4,4.0,"It could be so good, but it isn’t","Current Employee, more than 3 years",Fast Paced. Endless challenges. Inclusive envi...,The biggest perk of the job provides no value ...,NaN,o,o,o,3.0,3.0,3.0,1.0,4.0,5.0,https://www.glassdoor.com/Reviews/Calgary-Flam...,"Mar 25, 2023",Assistant Manager,NaN


In [2]:
# List of relevant features
relevant_features = [
    'pros', 'cons', 'Recommend', 'Career Opportunities', 'Compensation and Benefits', 
    'Senior Management', 'Work/Life Balance', 'Culture & Values', 'Diversity & Inclusion', 
    'job', 'status'
]

# Fill null values with mode for categorical columns and median for numerical columns
def fill_nulls(df, features):
    for column in features:
        if df[column].dtype == 'object':
            df[column] = df[column].fillna(df[column].mode()[0])
        else:
            df[column] = df[column].fillna(df[column].median())
    return df

df_sampled = fill_nulls(df_sampled, relevant_features + ['rating'])  # Apply to relevant features and 'rating'

# Check for any remaining null values in relevant features
df_sampled[relevant_features + ['rating']].isnull().sum()


pros                         0
cons                         0
Recommend                    0
Career Opportunities         0
Compensation and Benefits    0
Senior Management            0
Work/Life Balance            0
Culture & Values             0
Diversity & Inclusion        0
job                          0
status                       0
rating                       0
dtype: int64

In [3]:
# Frequency encoding for categorical variables
def frequency_encoding(df, columns):
    df_copy = df.copy()  # Create a copy of the DataFrame to avoid the warning
    for column in columns:
        # Generate the frequency encoding (i.e., percentage frequency of each unique value)
        freq = df_copy[column].value_counts() / len(df_copy)
        # Map the original column with its frequency encoding
        df_copy[column] = df_copy[column].map(freq).astype('float32')  # Explicitly cast to float32 to avoid dtype issues
    return df_copy

# Apply frequency encoding to the relevant non-numeric columns
df_sampled = frequency_encoding(df_sampled, relevant_features)

# Downcast again to save memory
df_sampled = downcast(df_sampled)


In [4]:
# Define X and y
X = df_sampled[relevant_features]  # Use only relevant features for X
y = df_sampled['rating']  # Target variable

# Check for non-numeric columns in X
non_numeric_columns_in_X = X.select_dtypes(include=['object', 'category']).columns
print("Non-numeric columns in X:", non_numeric_columns_in_X)

# Print the shapes of X and y
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Non-numeric columns in X: Index([], dtype='object')
Shape of X: (9901889, 11)
Shape of y: (9901889,)


In [5]:
from sklearn.model_selection import train_test_split

# Function to sample data
def sample_data(X, y, sample_size):
    if isinstance(sample_size, float):
        if 0 < sample_size < 1.0:
            return train_test_split(X, y, test_size=0.2, train_size=sample_size, random_state=42)
        elif sample_size == 1.0:
            return train_test_split(X, y, test_size=0.2, train_size=None, random_state=42)  # Fix for 1.0 sample size
        else:
            raise ValueError("sample_size as float must be in the range (0.0, 1.0) or equal to 1.0.")
    elif isinstance(sample_size, int):
        if sample_size > len(X):
            raise ValueError(f"sample_size {sample_size} exceeds the number of available samples {len(X)}.")
        sampled_X = X.sample(n=sample_size, random_state=42)
        sampled_y = y.loc[sampled_X.index]
        return train_test_split(sampled_X, sampled_y, test_size=0.2, random_state=42)
    else:
        raise ValueError("sample_size must be a float or an integer")


In [6]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import numpy as np
import time
import psutil

# Function to calculate and return metrics for Decision Tree Regressor
def calculate_metrics(X_train, X_test, y_train, y_test):
    dt = DecisionTreeRegressor(random_state=42)

    # Define hyperparameters for RandomizedSearchCV
    param_distributions = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 20],
        'min_samples_leaf': [1, 2, 4, 10],
        'max_features': ['sqrt', 'log2'],
        'max_leaf_nodes': [50, 100, 200],
    }

    random_search = RandomizedSearchCV(dt, param_distributions, n_iter=20, cv=5, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1)

    start_time = time.time()
    start_cpu = psutil.cpu_percent(interval=None)
    random_search.fit(X_train, y_train)
    end_cpu = psutil.cpu_percent(interval=None)
    end_time = time.time()

    # Calculate time and CPU usage
    execution_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    y_pred = random_search.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Calculate the range of the target variable
    target_range = y_train.max() - y_train.min()

    # Calculate normalized RMSE (nRMSE)
    nrmse = rmse / target_range
    
    memory_usage_MB = X_train.memory_usage(deep=True).sum() / (1024 ** 2)
    normalized_time = execution_time / memory_usage_MB
    
    return {
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2,
        'nRMSE': nrmse,  # Normalized RMSE
        'Execution Time (Raw)': execution_time,  # Raw execution time
        'Normalized Time (s/MB)': normalized_time,  # Normalized execution time
        'Average CPU Usage': avg_cpu_usage
    }


In [7]:
import gc  # Garbage Collector

# Define sample sizes
sample_sizes = [1.0, 0.5, 0.25, 0.125, 100, 1000, 10000, 100000]  

# Initialize metrics storage
metrics_list = []
total_execution_time = 0
total_cpu_usage = 0
total_memory_usage_MB = 0

# Loop through each sample size
for size in sample_sizes:
    try:
        # Sample data based on the defined sizes
        X_train_sample, X_test_sample, y_train_sample, y_test_sample = sample_data(X, y, size)
        
        # Calculate metrics
        metrics = calculate_metrics(X_train_sample, X_test_sample, y_train_sample, y_test_sample)
        metrics['Sample Size'] = size
        metrics_list.append(metrics)

        # Call garbage collection after each iteration to free up memory
        gc.collect()

        # Accumulate total metrics
        total_execution_time += metrics['Execution Time (Raw)']
        total_cpu_usage += metrics['Average CPU Usage']
        total_memory_usage_MB += X_train_sample.memory_usage(deep=True).sum() / (1024 ** 2)

        print(f"Metrics for sample size {size}:")
        for key, value in metrics.items():
            print(f"{key}: {value}")
        print("-" * 50)

    except Exception as e:
        print(f"An error occurred for sample size {size}: {e}")


Metrics for sample size 1.0:
RMSE: 0.7505577874839998
MAPE: 0.22611994460554233
R2: 0.6392698915958201
nRMSE: 0.18763944687099995
Execution Time (Raw): 254.8960223197937
Normalized Time (s/MB): 0.6488608906422408
Average CPU Usage: 55.6
Sample Size: 1.0
--------------------------------------------------
Metrics for sample size 0.5:
RMSE: 0.7547422975372046
MAPE: 0.22672514317318548
R2: 0.6352363938062131
nRMSE: 0.18868557438430114
Execution Time (Raw): 147.2089138031006
Normalized Time (s/MB): 0.59957382331065
Average CPU Usage: 54.15
Sample Size: 0.5
--------------------------------------------------
Metrics for sample size 0.25:
RMSE: 0.7547980326068654
MAPE: 0.2277935582931157
R2: 0.6351825187929232
nRMSE: 0.18869950815171635
Execution Time (Raw): 64.8830246925354
Normalized Time (s/MB): 0.5285299942488046
Average CPU Usage: 55.85
Sample Size: 0.25
--------------------------------------------------
Metrics for sample size 0.125:
RMSE: 0.7560589940787635
MAPE: 0.22444372989328898
R2:

In [8]:
# Convert metrics to DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Calculate total metrics
total_avg_cpu_usage = total_cpu_usage / len(sample_sizes)
normalized_total_time = total_execution_time / total_memory_usage_MB

# Convert total execution time to minutes and seconds
total_minutes = int(total_execution_time // 60)
total_seconds = total_execution_time % 60

# Display total metrics
print(f"Total Execution Time for Entire Process (Raw): {total_minutes} minutes and {total_seconds:.2f} seconds")
print(f"Total Normalized Execution Time for Entire Process: {normalized_total_time:.8f} seconds per MB")
print(f"Total Average CPU Usage for Entire Process: {total_avg_cpu_usage:.2f}%")

# Display the metrics DataFrame
metrics_df


Total Execution Time for Entire Process (Raw): 8 minutes and 18.29 seconds
Total Normalized Execution Time for Entire Process: 0.60259130 seconds per MB
Total Average CPU Usage for Entire Process: 49.18%


,RMSE,MAPE,R2,nRMSE,Execution Time (Raw),Normalized Time (s/MB),Average CPU Usage,Sample Size
0,0.750558,0.226120,0.639270,0.187639,254.896022,0.648861,55.60,1.000
1,0.754742,0.226725,0.635236,0.188686,147.208914,0.599574,54.15,0.500
2,0.754798,0.227794,0.635183,0.188700,64.883025,0.528530,55.85,0.250
3,0.756059,0.224444,0.633963,0.189015,29.104610,0.474166,59.20,0.125
4,1.031450,0.289722,0.295438,0.257862,0.144577,36.442368,35.80,100.000
5,0.895519,0.277087,0.509538,0.223880,0.149997,3.780847,35.35,1000.000
6,0.782420,0.226060,0.591647,0.195605,0.233158,0.587701,48.65,10000.000
7,0.758058,0.227568,0.636587,0.189515,1.667464,0.420303,48.80,100000.000


Another iteration

In [9]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import numpy as np
import time
import psutil

# Function to calculate and return metrics for Decision Tree Regressor
def calculate_metrics(X_train, X_test, y_train, y_test):
    dt = DecisionTreeRegressor(random_state=42)

    # Define hyperparameters for RandomizedSearchCV
    param_distributions = {
        'max_depth': [None, 10, 20, 30, 40, 50, 60, 80, 100],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 10, 20],
        'max_features': ['sqrt', 'log2'],
        'max_leaf_nodes': [50, 100, 200, 300],
    }

    random_search = RandomizedSearchCV(dt, param_distributions, n_iter=20, cv=5, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1)

    start_time = time.time()
    start_cpu = psutil.cpu_percent(interval=None)
    random_search.fit(X_train, y_train)
    end_cpu = psutil.cpu_percent(interval=None)
    end_time = time.time()

    # Calculate time and CPU usage
    execution_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    y_pred = random_search.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Calculate the range of the target variable
    target_range = y_train.max() - y_train.min()

    # Calculate normalized RMSE (nRMSE)
    nrmse = rmse / target_range
    
    memory_usage_MB = X_train.memory_usage(deep=True).sum() / (1024 ** 2)
    normalized_time = execution_time / memory_usage_MB
    
    return {
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2,
        'nRMSE': nrmse,  # Normalized RMSE
        'Execution Time (Raw)': execution_time,  # Raw execution time
        'Normalized Time (s/MB)': normalized_time,  # Normalized execution time
        'Average CPU Usage': avg_cpu_usage
    }


In [10]:
import gc  # Garbage Collector

# Define sample sizes
sample_sizes = [1.0, 0.5, 0.25, 0.125, 100, 1000, 10000, 100000]  

# Initialize metrics storage
metrics_list = []
total_execution_time = 0
total_cpu_usage = 0
total_memory_usage_MB = 0

# Loop through each sample size
for size in sample_sizes:
    try:
        # Sample data based on the defined sizes
        X_train_sample, X_test_sample, y_train_sample, y_test_sample = sample_data(X, y, size)
        
        # Calculate metrics
        metrics = calculate_metrics(X_train_sample, X_test_sample, y_train_sample, y_test_sample)
        metrics['Sample Size'] = size
        metrics_list.append(metrics)

        # Call garbage collection after each iteration to free up memory
        gc.collect()

        # Accumulate total metrics
        total_execution_time += metrics['Execution Time (Raw)']
        total_cpu_usage += metrics['Average CPU Usage']
        total_memory_usage_MB += X_train_sample.memory_usage(deep=True).sum() / (1024 ** 2)

        print(f"Metrics for sample size {size}:")
        for key, value in metrics.items():
            print(f"{key}: {value}")
        print("-" * 50)

    except Exception as e:
        print(f"An error occurred for sample size {size}: {e}")


Metrics for sample size 1.0:
RMSE: 0.739197439405154
MAPE: 0.21759379475298263
R2: 0.6501071811630184
nRMSE: 0.1847993598512885
Execution Time (Raw): 265.91571021080017
Normalized Time (s/MB): 0.6769125033527258
Average CPU Usage: 56.15
Sample Size: 1.0
--------------------------------------------------
Metrics for sample size 0.5:
RMSE: 0.7436779945890363
MAPE: 0.21817131246650653
R2: 0.6458526606429578
nRMSE: 0.18591949864725907
Execution Time (Raw): 151.86959171295166
Normalized Time (s/MB): 0.6185565085396613
Average CPU Usage: 58.95
Sample Size: 0.5
--------------------------------------------------
Metrics for sample size 0.25:
RMSE: 0.7490989657308125
MAPE: 0.22519135504750276
R2: 0.6406707950940107
nRMSE: 0.18727474143270312
Execution Time (Raw): 67.64465522766113
Normalized Time (s/MB): 0.551025933329389
Average CPU Usage: 60.3
Sample Size: 0.25
--------------------------------------------------
Metrics for sample size 0.125:
RMSE: 0.749294937554821
MAPE: 0.22196164944188734
R

In [11]:
# Convert metrics to DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Calculate total metrics
total_avg_cpu_usage = total_cpu_usage / len(sample_sizes)
normalized_total_time = total_execution_time / total_memory_usage_MB

# Convert total execution time to minutes and seconds
total_minutes = int(total_execution_time // 60)
total_seconds = total_execution_time % 60

# Display total metrics
print(f"Total Execution Time for Entire Process (Raw): {total_minutes} minutes and {total_seconds:.2f} seconds")
print(f"Total Normalized Execution Time for Entire Process: {normalized_total_time:.8f} seconds per MB")
print(f"Total Average CPU Usage for Entire Process: {total_avg_cpu_usage:.2f}%")

# Display the metrics DataFrame
metrics_df


Total Execution Time for Entire Process (Raw): 8 minutes and 37.56 seconds
Total Normalized Execution Time for Entire Process: 0.62589290 seconds per MB
Total Average CPU Usage for Entire Process: 47.23%


,RMSE,MAPE,R2,nRMSE,Execution Time (Raw),Normalized Time (s/MB),Average CPU Usage,Sample Size
0,0.739197,0.217594,0.650107,0.184799,265.915710,0.676913,56.15,1.000
1,0.743678,0.218171,0.645853,0.185919,151.869592,0.618557,58.95,0.500
2,0.749099,0.225191,0.640671,0.187275,67.644655,0.551026,60.30,0.250
3,0.749295,0.221962,0.640483,0.187324,29.908543,0.487263,54.75,0.125
4,1.073028,0.299306,0.237491,0.268257,0.133632,33.683594,31.55,100.000
5,0.895519,0.277087,0.509538,0.223880,0.143096,3.606911,34.15,1000.000
6,0.785826,0.228076,0.588084,0.196457,0.216870,0.546647,40.40,10000.000
7,0.755541,0.224465,0.638996,0.188885,1.723957,0.434543,41.60,100000.000
